## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [14:27,  8.64it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:39<00:00, 188.05it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Homeboy,Indiana,"Love charts, Supply & Demand and now the OTC m...",2015-10-13 16:08:09,1360,989,6236,False,2020-08-13 10:46:14,$ATWT has a huge chart correction yesterday an...,None,Twitter for iPhone,False
0,❌ Romulus ❌,❌,#MAGA Hardworking American living in U.S. wher...,2013-01-30 09:14:18,3312,4967,26367,False,2020-08-13 10:46:13,@DSteinhardtEsq @Aikens_Josh Election is a cor...,None,Twitter Web App,False
0,Kent Page,UNICEF Headquarters New York,"🇺🇳UNICEF Strategic Communication Advisor, Emer...",2009-03-14 00:20:10,88564,36813,19321,True,2020-08-13 10:46:12,This little one proudly wears their mask in th...,"[Covid19, Bangladesh]",Twitter Web App,False
0,DhakaTribune,"Dhaka, Bangladesh",Dhaka Tribune is the fastest growing English d...,2012-08-14 09:01:02,316940,190,55,False,2020-08-13 10:46:07,"Sweden, #UNFPA help #Bangladesh in #Covid19 re...","[UNFPA, Bangladesh, Covid19]",Twitter for Android,False
0,💧BlotReport,Australia,"Research scientist, progressive, for evidence ...",2017-01-08 01:37:20,2579,1959,27923,False,2020-08-13 10:46:04,"Hey brassica, @realDonaldTrump, there are 169,...",[COVID19],Twitter Web App,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (117601, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 117601 all tweets: 125101


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (125101, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)